## Camera Selection

In [1]:
import scipy.io as spio
#from __future__ import print_function

import numpy as np
import os

from IPython.display import clear_output
from IPython import display

import matplotlib.pyplot as plt
#import cv2 as cv
%matplotlib inline

In [2]:
import string
import random
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.distributions import Categorical
import time, math
 
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

use_cuda = torch.cuda.is_available()

if use_cuda:
    print ('CUDA is available')
#use_cuda=False   #uncomment this if you dont want to use cuda variables

CUDA is available


In [3]:
import sys
import os
import gc
import collections
# import hickle as hkl
# import ttictoc as tt

sys.path.insert(0, '../data/')
import get_pid_train_test as db
import auxiliary as af

In [4]:
# sys.path.insert(0,'../py-MDNet/modules')
# from sample_generator import *
import pandas as pd

### For Validation

In [5]:
# Test function
def plot_current_state(ped, c,fno):
    # load image for current location
    img,bb = load_image(ped,c,fno,db_no)

    dpi = 80.0
    #figsize = (img.size[0]/dpi, img.size[1]/dpi)
    figsize = (img.shape[0]/dpi, img.shape[1]/dpi)
    fig = plt.figure(frameon=False, figsize=figsize, dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    # get image and rect handle
    imAX = ax.imshow(img, aspect='normal')
    rect = plt.Rectangle(tuple(bb[0,:2]),bb[0,2],bb[0,3], 
        linewidth=3, edgecolor="#ff0000", zorder=1, fill=False)
    ax.add_patch(rect)

    plt.pause(.01)
    plt.draw()
    #fig.savefig(os.path.join(savefig_dir,'0000.jpg'),dpi=dpi)
    
    return imAX, rect
    
def plot_second(ped,c,curr_frame, imAX,rect):
    img,bb =  load_image(ped,c,curr_frame,db_no)
    #if np.array(img).shape[0] > 0:
    if img != []:
        imAX.set_data(img)
    #print (bb)

    #if bb.shape[0] > 0:
    if bb != []:
        rect.set_xy(bb[0,:2])
        rect.set_width(bb[0,2])
        rect.set_height(bb[0,3])
        print ('Correct camera')
    elif c!= num_camera-1:
        print ('Wrong camera')

    
    display.display(plt.gcf())
    plt.pause(1)
    plt.draw()
    #fig.savefig(os.path.join(savefig_dir,'%04d.jpg'%(i)),dpi=dpi)

def get_reward_gt(ped, curr_frame, c):
    y = afc.find_target_camera(ped,curr_frame)
    # get reward (give reward at end of episode)
    if y == num_camera-1 and y == c:
        reward = 0
    elif y == c:
        reward = 1
    else:
        reward = -1
        
    return reward,y

def get_next_step(ped,c,curr_frame, state):
    # update current state and history
    ispresent,this_state = get_state_vector(ped, c,curr_frame)
    if ispresent:
        next_state = this_state
    else:
        # use previous state
        next_state = state
    
    # get correct label from ground truth
    reward,y = get_reward_gt(ped, curr_frame,c)

    return next_state,reward,y,ispresent

def test_func(pTest, iloc='first', eloc='last', fixLoc=-1, isdebug=0, req_inc=1):
    policy_net.eval()
    rsT,accT = [],[]
    Qvalues = []
    numTrAllP = []
    pred = []
    gt = []
    
    for p in range(pTest.shape[0]): 
        reward_sum = 0
        accP = []
        inc = 1
        aaa = 1
        Qval_1p= []
        numTr = 0
        
        # load p'th person data
        ped = np.copy(pTest[p])
        # camera index and frame index starts from zero
        ped[:,0] -= 1
        ped[:,1] -= 1
        
        # Initialize with current state with start frame
        if iloc == 'first':
            startIDX = 0
        elif iloc == 'rand':
            startIDX = np.random.randint( 0,ped.shape[0]-20 )
        elif iloc == 'fix':
            startIDX = fixLoc
        if startIDX > ped.shape[0]:
            continue
        myPos = ped[startIDX,0:]
        print ('Initial position: ',myPos)
        
        curr_camera = myPos[0]
        curr_frame = myPos[1]
        
        # Initialize history variable (one-hot encoding)
        ch = np.zeros((h_len,duke_cam))
        occ_len = 0.01
        # Make initial state
        x_t,c_t,te_tau,r_t = make_state_vector(ped, curr_camera,curr_frame,ch,occ_len)
        prev_rt = r_t[0:4]
        #print (state.size())
        num_steps = 0
        prev_camera = curr_camera
        count_curr_c = 0
        
        if render: # show current location
            plot_current_state(ped, curr_camera,curr_frame)

            display.clear_output(wait=True)
            display.display(plt.gcf())

        # select an action from the current state
        hidden, cell = enc(torch.from_numpy(ch).float().cuda().unsqueeze(1))
        #print (x_t.size(),h_t.size(),enc_history.size())
        state_xt = torch.cat([x_t, te_tau], dim=1)
        state = torch.cat([state_xt, hidden[1,].detach()], dim=1)
        
        
        while(curr_frame <= ped[-1,1]): # alltime-6):
            
            if use_cuda:
                state_in = Variable(state)
                value_c = policy_net(state_in)
            else:
                state_in = Variable(state)
                value_c = policy_net(state_in)
                
            # Only exploitation for testing
            camera_index = torch.argmax(value_c)
            c = camera_index.detach().cpu().numpy()
            
            occ_max_val = 12000000
            aaa += 1
            if aaa > 1 and occ_len > occ_max_val:
                c = c #np.array(num_camera-1)
            if occ_len > occ_max_val and aaa%50 == 0:
                aaa = 1
                c = np.array(np.random.randint(num_camera))

            # find target for the next frame
            curr_frame += fpsc
            num_steps += 1
            
            # get correct label from ground truth
            reward,y = get_reward_gt(ped, curr_frame,c)
            #if req_inc:
            if inc==1 and y!=num_camera-1:
                # inside a camera
                if req_inc:
                    accP.append((y,y))
                    c = y
                else:
                    accP.append((y,c.item(0)))
            elif inc==0 and y==c.item(0) and y!=num_camera-1:
                # transitioning to second camera
                accP.append((y,c.item(0)))
                inc = 1
                numTr += 1
            elif inc==1 and y==num_camera-1:
                # moving out of a camera FOV
                inc = 0
                accP.append((y,c.item(0)))
            else:
                # Making transition
                accP.append((y,c.item(0)))
                #print ('Another case',y,c.item(0))
                    
            #else:
            #    accP.append((y,c.item(0)))
            
            # get the current bounding box
            bbox = ped[ np.logical_and(ped[:,0]==c,ped[:,1]==curr_frame),2:]
            if bbox.shape[0] > 0: # and np.random.rand < 0.95:
                bbox = bbox[0]
                rt = np.zeros((8))
                rt[0] = bbox[0]/imw -(np.random.rand()-0.5)/100
                rt[1] = bbox[1]/imh -(np.random.rand()-0.5)/100
                rt[2] = bbox[2]/imw -(np.random.rand()-0.5)/100
                rt[3] = bbox[3]/imh -(np.random.rand()-0.5)/100
                rt[4] = rt[0] - prev_rt[0] if occ_len < 0.2 else 0
                rt[5] = rt[1] - prev_rt[1] if occ_len < 0.2 else 0
                rt[6] = rt[2] - prev_rt[2] if occ_len < 0.2 else 0
                rt[7] = rt[3] - prev_rt[3] if occ_len < 0.2 else 0
                curr_camera = c
                
                # make next_state vector
                this_cam = afc.make_one_hot_camera(curr_camera)
                x_t = np.concatenate((this_cam, rt.ravel()))
                x_t[x_t==0] = -10
                x_t[x_t==1] = 10
                x_t = x_t.reshape(1,-1)
                if use_cuda:
                    x_t = torch.from_numpy(x_t).float().cuda()
                
                
                ispresent = 1
                prev_rt = rt[0:4]
                    
            else:
                ispresent = 0
                
            #############################################################################
            if c != num_camera-1:
                #bbox_pt = ped[ np.logical_and(ped[:,0]==c,ped[:,1]==curr_frame),2:]
                #if bbox_pt_n.shape[0] > 0:
                if ispresent:
                    pred.append([c+1,curr_frame+1,test_pids[p]+1,bbox[0],bbox[1],bbox[2],bbox[3]])
                #else:
                #    pred.append([c+1,curr_frame,p,np.array([0,0,0,0])])
            if y != num_camera-1:
                bbox_gt = ped[ np.logical_and(ped[:,0]==y,ped[:,1]==curr_frame),2:]
                gt.append([y+1,curr_frame+1,test_pids[p]+1,bbox_gt[0][0],bbox_gt[0][1],bbox_gt[0][2],bbox_gt[0][3]])
            
            #############################################################################
                
            # count the time of prev_camera selection
            if ispresent:
                occ_len = 0.01
            else:
                occ_len += 1
            #hcount = np.array(-occ_max_val + (occ_len/500)*(occ_max_val-(-occ_max_val)))
            hcount = np.array(10*np.log(occ_len))
            
            # update current state and history
            ch[1:,] = ch[0:-1,]
            ch[0,0:num_camera] = afc.make_one_hot_camera(c)
            ch[0,num_camera:] = 0
            this_cam = afc.make_one_hot_camera(c)
            c_t = this_cam.reshape(1,-1)
            
            if use_cuda:
                c_t = torch.from_numpy(c_t).float().cuda()
                te_tau = torch.from_numpy(hcount.reshape(1,-1)).float().cuda()
            else:
                c_t = torch.from_numpy(c_t).float()
                te_tau = torch.from_numpy(hcount.reshape(1,-1)).float()
                
            if isdebug:
                print ( np.where(rt.ravel()))
                print ( np.where(ch))
                print (c, curr_frame)
                print ('isPresent', ispresent)
                
            # make next_state vector
            hidden, cell = enc(torch.from_numpy(ch).float().cuda().unsqueeze(1))
            #print (x_t.size(),h_t.size(),enc_history.size())
            next_state_xt = torch.cat([x_t, te_tau], dim=1)
            next_state = torch.cat([next_state_xt, hidden[1,].detach()], dim=1)
            
            # store current reward
            reward_sum += reward
            Qval_1p.append((list(value_c.detach().cpu().numpy()[0]),hcount.ravel()[0],reward,False,y,c,state.detach().cpu().numpy()))
                        
            #state = next_state
            #state_xt = next_state_xt
            state = next_state #torch.cat([state_xt, enc_history], dim=1)
            prev_camera = c
            
            if render:
                plot_second()
            if eloc != 'last':
                if num_steps > eloc:
                    break
            
        # stack episodic reward 
        Qvalues.append((np.stack(Qval_1p)))
        rsT.append((reward_sum,num_steps))
        accT.append(accP)
        numTrAllP.append(numTr)
        
    return rsT, accT, Qvalues, numTrAllP, pred,gt

### Initialize hyper-parameters

In [6]:
learning_rate = 1e-4
batch_size = 1500
replay_memory_size = 20000
#epsilon = 0.1
gamma = 0.99

resume = False # resume from previous checkpoint
render = False
eps = np.finfo(np.float32).eps.item()


In [8]:
# load dataset
db_no = 2   # 1-4: NLPR-Set1-4, 5: DukeMTMC, (different file for CityFlow and WNMF)

# load test set for current data set
[pTest,num_camera,alltime,fps] = db.get_pid(set_no=db_no, train_flag='test')
num_camera += 1  # occlusion is also considered as a FOV
pTest = np.array(pTest)

if db_no == 5:
    fpsc = 6
    (imw,imh) = (1920,1080)
elif db_no == 6:
    fpsc = 1
    (imw,imh) = (1920,1080)
elif db_no < 5:
    fpsc = 2
    (imw,imh) = (320,240)

Total number of person in data set:  (1, 255)


In [9]:
if db_no == 5:
    tvmat = spio.loadmat('/home/anils/8tb/hpc-storage/dukeMTMC/ground_truth/trainval.mat')
    camGTT = tvmat['trainData'].astype(np.int)
    camGT = camGTT[:,[0,2,3,4,5,6]]
    camGT[:,0]-=1
    camGT[:,1]-=1
    # camGT = matlab.double(camGT)
elif db_no < 5:
    camGTT = pd.read_csv(os.path.expanduser('~/8tb/hpc-storage/nlpr/annotation_files/annotation/Dataset' + str(db_no) + '/fullgt.csv'), header=None).values.astype(np.int)[:,:7]
    tmp = np.copy(camGTT[:,1])
    camGTT[:,1] = camGTT[:,2]
    camGTT[:,2] = tmp
    camGT = camGTT[:,[0,2,3,4,5,6]]

    # In NLPR GT, frames already start with 0, only cam needs to be decremented
    camGT[:,0]-=1
    # camGT[:,1]-=1
    # camGT = matlab.double(camGT)
else:
    print ('Wrong dataset for this file.')
print('Shape of full ground truth', camGT.shape)


Shape of full ground truth (88827, 6)


In [10]:
numEpoch = 100000
d = 10
region_size = (d,d)

h_len = 10

# Load auxiliary functions using an object
afc = af.AuxiliaryFunction(num_camera=num_camera, d=d, h_len=h_len)

## Policy Model

In [11]:
# Initialize required parameters
lstm_size = 256
hidden_size1 = 4096
hidden_size2 = 2048
hidden_size3 = 256

input_size = lstm_size + num_camera+ 4*2 +1

# Required network
class NextCamera(nn.Module):
    def __init__(self):
        super(NextCamera, self).__init__()
        
        # make decoder layers
        self.fch1 = nn.Linear(input_size, hidden_size1)
        self.fch2 = nn.Linear(hidden_size1, hidden_size2)
        self.fch3 = nn.Linear(hidden_size2, hidden_size3)
        self.fco = nn.Linear(hidden_size3, num_camera)
        
        # Activation function 
        self.tanh = nn.Tanh() #ReLU()
        self.relu = nn.ReLU() #ReLU()
        #self.linear = nn.Linear() 
        
    def forward(self, x):
        
        x = self.relu(self.fch1(x))
        x = self.relu(self.fch2(x))
        x = self.relu(self.fch3(x))
        x = self.fco(x)
            
        return x # nn.functional.softmax(x)

In [12]:
# build the model
if use_cuda:
    policy_net = NextCamera().float().cuda()
    criterion = nn.MSELoss().cuda()
    
else:
    policy_net = NextCamera().float()
    criterion = nn.MSELoss()
    
if use_cuda:
    target_net = NextCamera().cuda()
    target_net.float().cuda()
target_net.load_state_dict(policy_net.state_dict())

# use ADAM as optimizer since we can load the whole data to train
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)

In [13]:
def backward_network(replay_memory_pos,pos_prob, replay_memory_neg, replay_memory_cx, update_criteria):

    # sample random minibatch
    minibatch_pos = random.choices(replay_memory_pos, k=min(len(replay_memory_pos), 500), weights=pos_prob)
    #minibatch_pos = random.sample(replay_memory_pos, min(len(replay_memory_pos), 300)) #int(batch_size/3)))
    minibatch_posneg = minibatch_pos + random.sample(replay_memory_neg, min(len(replay_memory_neg), 500)) # int(batch_size/3)))
    minibatch = minibatch_posneg + random.sample(replay_memory_cx, min(len(replay_memory_cx), 500)) #int(batch_size/3)))
    
    # unpack minibatch
    #state_xt = tuple(d[0] for d in minibatch)
    state = torch.cat(tuple(d[0] for d in minibatch))
    #prev_ch = tuple(d[1] for d in minibatch)
    action = torch.cat(tuple(d[1] for d in minibatch))
    reward = torch.cat(tuple(d[2] for d in minibatch))
    #next_state_xt = tuple(d[4] for d in minibatch)
    next_state = torch.cat(tuple(d[3] for d in minibatch))
    #ch = tuple(d[5] for d in minibatch)
    
    # num samples of different categories
    numRew = torch.stack([torch.sum(reward>=0.2),torch.sum(reward==-1),torch.sum(reward==0.01)]).data.cpu().numpy()
    
    # get output for the next state
    next_output = target_net(next_state)

    # set y_j to r_j for terminal state, otherwise to r_j + gamma*max(Q)
    y = torch.cat(tuple(reward[i] if minibatch[i][4] \
                        else reward[i] + gamma * torch.max(next_output[i]) \
                        for i in range(len(minibatch))))

    # extract Q-value
    q_value = torch.sum(policy_net(state) * action, dim=1)

    # PyTorch accumulates gradients by default, so they need to be reset in each pass
    optimizer.zero_grad()

    # returns a new Tensor, detached from the current graph, the result will never require gradient
    y = y.detach()

    #print (y, q_value)
    # calculate loss
    loss = criterion(q_value, y)

    # do backward pass
    loss.backward()
    optimizer.step()

    # copy weights from policy_net to target_net
    if update_criteria == 10:
        target_net.load_state_dict(policy_net.state_dict())
        update_criteria = 0
    update_criteria += 1
    
    return loss.data,numRew,update_criteria

## AE Model

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = src #self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size, dim]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size, dim]
        
        embedded = input #self.dropout(self.embedding(input))
        #embedded[np.arange(embedded.size),a] = 1
        
        #embedded = [1, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.1):
        
        #src = [src len, batch size, dim]
        #trg = [trg len, batch size, dim]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output #.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [15]:
duke_cam = 9
INPUT_DIM = duke_cam
OUTPUT_DIM = duke_cam
ENC_EMB_DIM = duke_cam
DEC_EMB_DIM = duke_cam
HID_DIM = 256
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT).float().cuda()
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT).float().cuda()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).float().to(device)
criterion_ae = nn.CrossEntropyLoss()

In [16]:
# load encoder model
#enc.load_state_dict(torch.load('enc-model_manyDB_state64.pt'))
#enc.load_state_dict(torch.load('enc-model_manyDB.pt'))
#enc.eval()
#dec.load_state_dict(torch.load('dec-model_manyDB_state64.pt'))
#dec.eval()
model.load_state_dict(torch.load('../eccv2020/tut1-model_duke_lstmSize128_manyDB_2.pt'))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9, 9)
    (rnn): LSTM(9, 256, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(9, 9)
    (rnn): LSTM(9, 256, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=9, bias=True)
    (dropout): Dropout(p=0.5)
  )
)

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Run episodes

In [18]:
reward_sum = 0
running_reward = None
xs,rs,cprs = [],[],[]
episode_number = 0
episode_durations = []
episode_reward = []
validation_reward= []
replay_memory_pos = []
pos_prob = []
replay_memory_neg = []
replay_memory_cx = []
M = np.zeros((num_camera,num_camera))

In [19]:
def make_state_vector(ped, curr_camera,curr_frame, ch,occ_len):
    numSamples = 30
    overlap_thres = [0.9, 1]
        
    # read image
    img,bbox,p = afc.load_image(ped,curr_camera,curr_frame,db_no)
    #imw, imh = (320,240) #img.size
    hc = np.array(10*np.log(occ_len))
    
    if p:
        rt = np.zeros((8))
        rt[0] = bbox[0]/imw -(np.random.rand()-0.5)/100
        rt[1] = bbox[1]/imh -(np.random.rand()-0.5)/100
        rt[2] = bbox[2]/imw -(np.random.rand()-0.5)/100
        rt[3] = bbox[3]/imh -(np.random.rand()-0.5)/100
        rt[4] = 0
        rt[5] = 0
        rt[6] = 0
        rt[7] = 0
        #print (np.where(rt.ravel()))
        
        # make next_state vector
        #this_cam = afc.make_one_hot_camera(curr_camera)
        #state = np.concatenate((this_cam, rt.ravel()))
        #state = np.concatenate((state, hc.ravel()))
        #state = np.concatenate((state, ch.ravel()))
        #state = state.reshape(1,-1)
        #state[state==0] = -10
        #state[state==1] = 10
        
        # make next_state vector
        this_cam = afc.make_one_hot_camera(curr_camera)
        xt = np.concatenate((this_cam, rt.ravel()))
        xt[xt==0] = -10
        xt[xt==1] = 10
        xt = xt.reshape(1,-1)
        
        # make history vector
        c_t = this_cam.reshape(1,-1)
        
        if use_cuda:
            xt = torch.from_numpy(xt).float().cuda()
            c_t = torch.from_numpy(c_t).float().cuda()
            hc = torch.from_numpy(hc.reshape(1,-1)).float().cuda()
        else:
            xt = torch.from_numpy(xt).float()
            c_t = torch.from_numpy(c_t).float()
            hc = torch.from_numpy(hc.reshape(1,-1)).float()
    else:
        print ('Target is not present in ',c,curr_frame)
        xt,h_t = [],[]
    
    return xt,c_t,hc,rt #p,state,rt

def append_reward(rs,num_steps):
    if len(rs) > 0:
        # stack episodic reward 
        epR = np.vstack(rs)
        rs = []

        # append the episodic reward
        #episode_number += 1
        #episode_durations.append(num_steps)
        reward_stat = [num_steps,np.std(epR),np.sum(epR)]
        episode_reward.append(reward_stat)
    
    return rs

def reinit_ae(ch):
    # Initialize history variable (one-hot encoding)
    if use_cuda:
        ch = torch.from_numpy(ch).float().cuda()
        enc_h = torch.zeros(1,lstm_size).float().cuda()
        enc_c = torch.zeros(1,lstm_size).float().cuda()
    else:
        enc_h = torch.zeros(1,lstm_size).float()
        enc_c = torch.zeros(1,lstm_size).float()
        
    # encode whole camera history
    for i in range(seq_len-1,-1,-1):
        #print (ch[i,:])
        x = ch[i,:].view(1,-1)
        h_lstm,enc = ae_enc((enc_h,enc_c), x)
        (enc_h,enc_c) = h_lstm

    return h_lstm,enc

In [20]:
epoch = 0
occ_max_val = 100

In [21]:
EpData = []
allEpData = []
numRew=[]

numUpdateRew=[]
update_criteria = 0
episode_count = 0
steps_count = 0
initialEpsilon = 0.4
finalEpsilon = 0.01
epsilon = initialEpsilon

In [22]:
trExplored = {}
for i in range(num_camera):
    for j in range(num_camera):
        trExplored[str(i)+'-'+str(j)] = []

In [23]:
# tt.tic()

In [24]:
def bb_intersection_over_union(boxA, boxB):
    # convert to x1,y1,x2,y2
    boxA[2] = boxA[2] + boxA[0]
    boxA[3] = boxA[3] + boxA[1]
    boxB[2] = boxB[2] + boxB[0]
    boxB[3] = boxB[3] + boxB[1]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

def find_nearest_box(c,frame_no, prev_box):
    # find all bounding boxes
    all_dets = np.copy(camGT[np.logical_and(camGT[:,0]==c, camGT[:,1]==frame_no), 2:])
    
    #print (all_dets, prev_box)
    
    # find the nearest box
    maxIOU = -1
    box = np.array([])
    for i in range(len(all_dets)):
        this_box = np.copy(all_dets[i])
        iou = bb_intersection_over_union(this_box, prev_box)
        #print (iou, this_box)
        if iou > maxIOU and iou > 0.4:
            maxIOU = iou
            box = np.copy(all_dets[i])
            
    return box

In [25]:

def discount_rewards(r,c):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = [] #np.zeros_like(r)
    running_add = 0
    pivot = c[-1]
    for t in reversed(range(0, len(r))):
        if r[t] != 0: 
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
            pivot = c[t]
        if c[t] == pivot:
            running_add = running_add * gamma + r[t]
            #discounted_r[t] = running_add
            discounted_r.append(running_add)
        else:
            running_add = running_add * gamma + 0
            discounted_r.append(0)
            
        
    return discounted_r

In [26]:
def remove_duplicates(arr): 
    final_list = [] 
    gt_tr = []
    final_list.append(arr[0])     
    for i in range(1,arr.shape[0]): 
        if arr[i] != arr[i-1]:
            final_list.append(arr[i])     
            if arr[i] != num_camera-1:
                gt_tr.append(arr[i])
    return final_list, gt_tr

In [ ]:
if db_no == 1:
    rand_PIDs = np.array([88,137,182,223,174,71,200,58,1,185,74,168,235,46,17,67,162,100, \
        192,113,140,6,94,81,11,181,159,78,147,22,127,115,68,143,59,212,217,161, \
        92,164,227,206,73,123,125,126,230,233,83,197,231,34,145,26,114,27,138,7,\
        151,8,48,120,166,215,35,85,64,111,156,225,65,171,153,25,45,207,219,178,\
        23,165,50,199,110,203,211,47,152,30,132,102,205,96,89,129,52,107,60,36,\
        216,167,87,149,208,116,213,57,41,214,12,175,163,15,173,144,134,86,194,82,\
        128,186,63,105,122,69,21,183,169,187,222,19,232,108,198,79,141,91,51,150,\
        53,77,5,119,39,33,170,84,229,180,133,40,188,139,54,121,158,55,42,10,9,124,\
        142,90,136,189,226,131,135,13,95,24,209,191,44,29,16,218,20,93,184,130,117,\
        49,204,3,112,146,195,109,62,37,98,154,72,99,157,234,103,56,106,31,66,172,202,\
        38,43,28,101,221,193,177,2,4,160,155,14,210,196,176,97,70,201,228,75,18,104,\
        179,118,148,190,224,220,32,61,80,76])
    test_pids = rand_PIDs[1::2]-1
elif db_no == 2:
    rand_PIDs = range(255)
    test_pids = rand_PIDs
elif db_no == 3:
    rand_PIDs = np.array([9,1,5,2,8,7,6,12,13,11,4,14,10,3])
    test_pids = rand_PIDs[1::2]-1
elif db_no == 4:
    rand_PIDs = np.array([1,46,17,6,11,22,34,26,27,7,8,48,35,25,45,23,47,30,36,41,12,15,21,19,5,39,\
            33,40,42,10,9,13,24,44,29,16,20,49,3,37,31,38,43,28,2,4,14,18,32])
    test_pids = rand_PIDs[1::2]-1
elif db_no == 5:
    randPIDs = spio.loadmat('../data/pALL_DukeDataset_rand.mat')  # load the random PIDs
    randPIDs = randPIDs['pALL']-1
    test_pids = randPIDs[0,1::2] 

In [27]:
if db_no == 1 or db_no == 2:
    policy_net.load_state_dict(torch.load('./models/policy_db1_semisup_gtBOX_5_841')['state_dict'])
if db_no == 3:
    policy_net.load_state_dict(torch.load('./models/policy_db3_semisup_gtBOX_5_1321')['state_dict'])
if db_no == 4:
    policy_net.load_state_dict(torch.load('./models/policy_db4_semisup_gtBOX_5_1521')['state_dict'])
if db_no == 5:
    policy_net.load_state_dict(torch.load('./models/policy_duke_semisup_gtBOX_5_16')['state_dict'])

In [28]:
policy_net.eval()
req_inc = 0
render = False
_,acc,_,numTR = test_func(pTest,iloc='fix',eloc='last', fixLoc=2, isdebug=0, req_inc=req_inc)

resMat = {}
resMat['MCT_pr'] = rp
resMat['MCT_gt'] = rg
sel_matrix = np.vstack(acc)
resMat['Sel_pr'] = sel_matrix[:,1]
resMat['Sel_gt'] = sel_matrix[:,0]
resMat['accAllP'] = acc
resMat['numTR'] = numTR

# experiment number, only ICT: exp1, ICT+SCT: exp2
if req_inc == 0:   # performing ICT+SCT
    expNo = 2
else:    # performing only ICT
    expNo = 1  
    
if db_no == 1:
    saveFilePath = './results/results_MM_db1_exp'+str(expNo)+'_gt.mat'
elif db_no == 2:
    saveFilePath = './results/results_MM_db2_exp'+str(expNo)+'_gt.mat'
elif db_no == 3:
    saveFilePath = './results/results_MM_db3_exp'+str(expNo)+'_gt.mat'
elif db_no == 4:
    saveFilePath = './results/results_MM_db4_exp'+str(expNo)+'_gt.mat'
elif db_no == 5:
    saveFilePath = './results/results_MM_duke_exp'+str(expNo)+'_gt.mat'


Initial position:  [ 0 19 76  2 21 52]
Initial position:  [  0  28 102   2  23  59]
Initial position:  [  0 158  89   2  26  53]
Initial position:  [  0 281 111   2  22  54]
Initial position:  [  0 313  65   2  25  53]
Initial position:  [  0 422 222 174  25  65]
Initial position:  [  0 512 218 177  23  61]
Initial position:  [  0 658 104   1  21  59]
Initial position:  [  0 697  40   2  24  57]
Initial position:  [  0 818  71   1  23  56]
Initial position:  [  0 915  71   1  21  53]
Initial position:  [   0 1198   43    2   20   61]
Initial position:  [   0 1661   76    2   29   61]
Initial position:  [   0 1925  168   29   29   56]
Initial position:  [   0 2088   34    2   25   57]
Initial position:  [   0 2393   68    2   27   62]
Initial position:  [   0 2700   88    2   25   55]
Initial position:  [   0 3680   46    2   23   60]
Initial position:  [   0 3712  104    2   24   58]
Initial position:  [   0 4022   59    3   28   61]
Initial position:  [   0 4114  125    2   21   54]
I

Initial position:  [   2 4565   77   30   34   78]
Initial position:  [   2 4596  281   42   33   89]
Initial position:  [   2 4642  186   35   25   87]
Initial position:  [   2 4888  277   45   39   71]
Initial position:  [   2 5115  275   44   36   80]
Initial position:  [   2 6110  282   43   34   94]
Initial position:  [   2 6133  279   40   35   92]
Initial position:  [   2 6243  273   42   43   92]
Initial position:  [   2 6325  282   44   35   83]
Initial position:  [   2 6672  266  108   51  118]
Initial position:  [   2 6859   79   10   22   72]
Initial position:  [   2 8144  287   48   26   90]
Initial position:  [   2 8266   76   21   24   63]
Initial position:  [   2 8630  284   42   29   80]
Initial position:  [   2 9108   92  110   51  122]
Initial position:  [   2 9141  273   77   41  101]
Initial position:  [   2 9753  279   38   34   86]
Initial position:  [    2 10214   249   103    58   121]
Initial position:  [    2 10405   108   111    61   120]
Initial position:  

In [32]:
tr_acc = 0
avg_tr_captured = []
A,P,R,F, ttr = [],[],[],[],[]
Fscore, Fscore_onlytr = [],[]
A_onlytr,P_onlytr,R_onlytr = [],[],[]
nfr = []
for i in range(len(acc)):
    print ('Person: ',i)
    gt = np.array([d[0] for d in acc[i]])
    pr = np.array([d[1] for d in acc[i]])
    g = gt #t[gt != num_camera-1]
    p = pr #r[gt != num_camera-1]

    dups,gt_tr = remove_duplicates(g)
    print ('Transitions: ', dups)
    print ('GT transitions: ', len(gt_tr))
    print ('Transitions captured: ', numTR[i])
    if len(gt_tr) != 0:
        avg_tr_captured.append((numTR[i],len(gt_tr)))
        contains_tr = 1
    else:
        print ('')
        contains_tr = 0
        #continue

    # plot transitions
#     afc.plot_color_transitions(p,g)
    # MCTA and number of frames
    if req_inc == 1:
        ac,pr,re,fr,tr = afc.compute_APRF_one_person_sct_ict(p,g)
    else:
        ac,pr,re,fr,tr = afc.compute_APRF_one_person_sct_ict(p,g)

    fs = 2*(pr*re)/(pr+re)
    if contains_tr == 1:
        A_onlytr.append(ac)
        P_onlytr.append(pr)
        R_onlytr.append(re)
        Fscore_onlytr.append(fs)
    A.append(ac)
    P.append(pr)
    R.append(re)
    F.append(fr)
    Fscore.append(fs) 
    ttr.append(tr)
    print ('A,P,R: ', ac,pr,re)
    f = afc.compute_num_frames(p,g)
    nfr.append(f)
    print ('Num frames: ', f)
    # Accuracy
    tacc = np.sum(g==p, dtype=np.float)/g.shape[0]
    tr_acc += tacc
    print ('Accuracy: ',tacc)
print (tr_acc/len(A))
print ('Average (only transitions) A,P,R', np.mean(A_onlytr),np.mean(P_onlytr),np.mean(R_onlytr))
print ('Average (all targets) A,P,R,F, ttr', np.mean(A),np.mean(P),np.mean(R),np.mean(F), ttr)
print ('Fscore (all targets, only transitions) ', np.mean(Fscore),np.mean(Fscore_onlytr))
print (np.sum(nfr))

Person:  0
Transitions:  [0, 3, 1, 3, 2, 3]
GT transitions:  2
Transitions captured:  0
A,P,R:  0.507181328545781 0.23283082077051925 0.5054545454545455
Num frames:  (597, 413)
Accuracy:  0.507181328545781
Person:  1
Transitions:  [0, 3, 1, 3, 2, 3]
GT transitions:  2
Transitions captured:  2
A,P,R:  0.8601462522851919 0.6326530612244898 0.9649805447470817
Num frames:  (392, 144)
Accuracy:  0.8601462522851919
Person:  2
Transitions:  [0, 3, 1, 3, 2, 3]
GT transitions:  2
Transitions captured:  2
A,P,R:  0.7960526315789473 0.57 0.75
Num frames:  (400, 172)
Accuracy:  0.7960526315789473
Person:  3
Transitions:  [0, 3, 1, 3]
GT transitions:  1
Transitions captured:  0
A,P,R:  0.8362760834670947 0.9150326797385621 0.611353711790393
Num frames:  (153, 13)
Accuracy:  0.8362760834670947
Person:  4
Transitions:  [0, 3, 1, 3, 2, 3]
GT transitions:  2
Transitions captured:  0
A,P,R:  0.5146788990825688 0.2391681109185442 0.5227272727272727
Num frames:  (577, 403)
Accuracy:  0.5146788990825688
Pe

GT transitions:  2
Transitions captured:  0
A,P,R:  0.46317512274959083 0.20821529745042494 0.49328859060402686
Num frames:  (706, 505)
Accuracy:  0.46317512274959083
Person:  66
Transitions:  [0, 3, 1, 3]
GT transitions:  1
Transitions captured:  0
A,P,R:  0.827217125382263 0.8782051282051282 0.5930735930735931
Num frames:  (156, 19)
Accuracy:  0.827217125382263
Person:  67
Transitions:  [0, 3, 1, 3]
GT transitions:  1
Transitions captured:  1
A,P,R:  0.8292682926829268 0.9013157894736842 0.5854700854700855
Num frames:  (152, 15)
Accuracy:  0.8292682926829268
Person:  68
Transitions:  [0, 3, 1, 3, 2, 3, 2, 3]
GT transitions:  3
Transitions captured:  3
A,P,R:  0.7738376674546887 0.5205479452054794 0.7475409836065574
Num frames:  (438, 210)
Accuracy:  0.7738376674546887
Person:  69
Transitions:  [1, 3, 0, 3]
GT transitions:  1
Transitions captured:  1
A,P,R:  0.9312977099236641 0.845360824742268 0.9647058823529412
Num frames:  (194, 30)
Accuracy:  0.9312977099236641
Person:  70
Transit

Num frames:  (50, 4)
Accuracy:  0.92
Person:  187
Transitions:  [2, 3, 2, 3, 2, 3]
GT transitions:  2
Transitions captured:  2
A,P,R:  0.6011904761904762 0.9468085106382979 0.5894039735099338
Num frames:  (94, 5)
Accuracy:  0.6011904761904762
Person:  188
Transitions:  [2, 3]
GT transitions:  0
Transitions captured:  0

A,P,R:  0.9090909090909091 1.0 0.9
Num frames:  (9, 0)
Accuracy:  0.9090909090909091
Person:  189
Transitions:  [2, 3, 2, 3]
GT transitions:  1
Transitions captured:  0
A,P,R:  0.33962264150943394 0.30952380952380953 0.6842105263157895
Num frames:  (42, 29)
Accuracy:  0.33962264150943394
Person:  190
Transitions:  [2, 3]
GT transitions:  0
Transitions captured:  0

A,P,R:  0.9838709677419355 0.9838709677419355 1.0
Num frames:  (62, 1)
Accuracy:  0.9838709677419355
Person:  191
Transitions:  [2, 3]
GT transitions:  0
Transitions captured:  0

A,P,R:  0.9777777777777777 0.9777777777777777 1.0
Num frames:  (45, 1)
Accuracy:  0.9777777777777777
Person:  192
Transitions:  [2

In [30]:
a = np.stack(avg_tr_captured)
print (np.mean(a[:,0]/a[:,1]))
print (sum(a[:,0])/sum(a[:,1]))

0.6220430107526881
0.5953307392996109


In [ ]:
# Perceptage transitions captured (avg of all targets/ avg transitions)
a = np.stack(avg_tr_captured)
avgTrPerTarget = np.mean(a[:,0]/a[:,1])
print (avgTrPerTarget)
avgTr = sum(a[:,0])/sum(a[:,1])
print (avgTr)
totalTr = sum(a[:,1])

In [ ]:
resMat['avgTr'] = avgTr
resMat['totalTr'] = totalTr

spio.savemat(saveFilePath, resMat)
print ('results saved in ', saveFilePath)

### Following function is to test on all checkpoints

In [33]:
from IPython.display import clear_output
def eval_policy():
    policy_net.eval()
    req_inc = 0
    render = False
    _,acc,_,numTR = test_func(pTest,iloc='fix',eloc='last', fixLoc=2, isdebug=0, req_inc=req_inc)
    tr_acc = 0
    avg_tr_captured = []
    A,P,R,F, ttr = [],[],[],[],[]
    Fscore, Fscore_onlytr = [],[]
    A_onlytr,P_onlytr,R_onlytr = [],[],[]
    nfr = []
    for i in range(len(acc)):
        print ('Person: ',i)
        gt = np.array([d[0] for d in acc[i]])
        pr = np.array([d[1] for d in acc[i]])
        g = gt #t[gt != num_camera-1]
        p = pr #r[gt != num_camera-1]

        dups,gt_tr = remove_duplicates(g)
        print ('Transitions: ', dups)
        print ('GT transitions: ', len(gt_tr))
        print ('Transitions captured: ', numTR[i])
        if len(gt_tr) != 0:
            avg_tr_captured.append((numTR[i],len(gt_tr)))
            contains_tr = 1
        else:
            print ('')
            contains_tr = 0
            #continue

        # plot transitions
    #     afc.plot_color_transitions(p,g)
        # MCTA and number of frames
        if req_inc == 1:
            ac,pr,re,fr,tr = afc.compute_APRF_one_person_sct_ict(p,g)
        else:
            ac,pr,re,fr,tr = afc.compute_APRF_one_person_sct_ict(p,g)

        fs = 2*(pr*re)/(pr+re)
        if contains_tr == 1:
            A_onlytr.append(ac)
            P_onlytr.append(pr)
            R_onlytr.append(re)
            Fscore_onlytr.append(fs)
        A.append(ac)
        P.append(pr)
        R.append(re)
        F.append(fr)
        Fscore.append(fs) 
        ttr.append(tr)
        print ('A,P,R: ', ac,pr,re)
        f = afc.compute_num_frames(p,g)
        nfr.append(f)
        print ('Num frames: ', f)
        # Accuracy
        tacc = np.sum(g==p, dtype=np.float)/g.shape[0]
        tr_acc += tacc
        print ('Accuracy: ',tacc)
    print (tr_acc/len(A))
    print ('Average (only transitions) A,P,R', np.mean(A_onlytr),np.mean(P_onlytr),np.mean(R_onlytr))
    print ('Average (all targets) A,P,R,F, ttr', np.mean(A),np.mean(P),np.mean(R),np.mean(F), ttr)
    print ('Fscore (all targets, only transitions) ', np.mean(Fscore),np.mean(Fscore_onlytr))
    print (np.sum(nfr))
    
    a = np.stack(avg_tr_captured)
    PCH_1 = np.mean(a[:,0]/a[:,1])
    PCH_2 = sum(a[:,0])/sum(a[:,1])
    print (PCH_1)
    print (PCH_2)
    
    return np.mean(A),np.mean(P),np.mean(R),np.mean(Fscore),PCH_1,PCH_2

In [34]:

all_metric_values = []
for epoch_i in range(301,1250,20):
    modelname = './models/policy_db1_semisup_gtBOX_5_' + str(epoch_i)
    print (modelname)
    # load model
    policy_net.load_state_dict(torch.load(modelname)['state_dict'])
    
    A,P,R,F1,PCH1,PCH2 = eval_policy()
    all_metric_values.append((epoch_i,A,P,R,F1,PCH1,PCH2))
    
    clear_output(wait=True)

./models/policy_db1_semisup_gtBOX_5_1241
Initial position:  [ 0 19 76  2 21 52]
Initial position:  [  0  28 102   2  23  59]
Initial position:  [  0 158  89   2  26  53]
Initial position:  [  0 281 111   2  22  54]
Initial position:  [  0 313  65   2  25  53]
Initial position:  [  0 422 222 174  25  65]
Initial position:  [  0 512 218 177  23  61]
Initial position:  [  0 658 104   1  21  59]
Initial position:  [  0 697  40   2  24  57]
Initial position:  [  0 818  71   1  23  56]
Initial position:  [  0 915  71   1  21  53]
Initial position:  [   0 1198   43    2   20   61]
Initial position:  [   0 1661   76    2   29   61]
Initial position:  [   0 1925  168   29   29   56]
Initial position:  [   0 2088   34    2   25   57]
Initial position:  [   0 2393   68    2   27   62]
Initial position:  [   0 2700   88    2   25   55]
Initial position:  [   0 3680   46    2   23   60]
Initial position:  [   0 3712  104    2   24   58]
Initial position:  [   0 4022   59    3   28   61]
Initial pos

Initial position:  [   2 3722  279   34   39   86]
Initial position:  [   2 3759  282   44   35   74]
Initial position:  [   2 4565   77   30   34   78]
Initial position:  [   2 4596  281   42   33   89]
Initial position:  [   2 4642  186   35   25   87]
Initial position:  [   2 4888  277   45   39   71]
Initial position:  [   2 5115  275   44   36   80]
Initial position:  [   2 6110  282   43   34   94]
Initial position:  [   2 6133  279   40   35   92]
Initial position:  [   2 6243  273   42   43   92]
Initial position:  [   2 6325  282   44   35   83]
Initial position:  [   2 6672  266  108   51  118]
Initial position:  [   2 6859   79   10   22   72]
Initial position:  [   2 8144  287   48   26   90]
Initial position:  [   2 8266   76   21   24   63]
Initial position:  [   2 8630  284   42   29   80]
Initial position:  [   2 9108   92  110   51  122]
Initial position:  [   2 9141  273   77   41  101]
Initial position:  [   2 9753  279   38   34   86]
Initial position:  [    2 10214

Num frames:  (625, 310)
Accuracy:  0.7663410969196093
Person:  41
Transitions:  [0, 3, 1, 3, 2, 3, 2, 3]
GT transitions:  3
Transitions captured:  2
A,P,R:  0.7316341829085458 0.45 0.8259493670886076
Num frames:  (580, 303)
Accuracy:  0.7316341829085458
Person:  42
Transitions:  [0, 3, 0, 3, 1, 3]
GT transitions:  2
Transitions captured:  1
A,P,R:  0.7842105263157895 0.6962962962962963 0.5340909090909091
Num frames:  (135, 41)
Accuracy:  0.7842105263157895
Person:  43
Transitions:  [0, 3, 1, 3, 2, 3]
GT transitions:  2
Transitions captured:  2
A,P,R:  0.7204861111111112 0.4131455399061033 0.6743295019157088
Num frames:  (426, 237)
Accuracy:  0.7204861111111112
Person:  44
Transitions:  [1, 3, 0, 3]
GT transitions:  1
Transitions captured:  1
A,P,R:  0.717815344603381 0.5488565488565489 1.0
Num frames:  (481, 217)
Accuracy:  0.717815344603381
Person:  45
Transitions:  [1, 3, 0, 3]
GT transitions:  1
Transitions captured:  1
A,P,R:  0.7634252539912917 0.6034063260340633 0.972549019607843

/home/anils/vedantb/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


A,P,R:  0.17013582642441347 0.14848429842076602 0.3695504856562006
Num frames:  (11018, 6618)
Accuracy:  0.17013582642441347
Person:  133
Transitions:  [2, 3]
GT transitions:  0
Transitions captured:  0

A,P,R:  0.9696969696969697 0.9696969696969697 1.0
Num frames:  (33, 1)
Accuracy:  0.9696969696969697
Person:  134
Transitions:  [2, 3, 2, 3]
GT transitions:  1
Transitions captured:  1
A,P,R:  0.7735849056603774 0.7735849056603774 0.9318181818181818
Num frames:  (53, 9)
Accuracy:  0.7735849056603774
Person:  135
Transitions:  [2, 3, 2, 3]
GT transitions:  1
Transitions captured:  1
A,P,R:  0.6818181818181818 0.6818181818181818 0.8571428571428571
Num frames:  (44, 9)
Accuracy:  0.6818181818181818
Person:  136
Transitions:  [2, 3]
GT transitions:  0
Transitions captured:  0

A,P,R:  0.9848484848484849 0.9848484848484849 1.0
Num frames:  (66, 1)
Accuracy:  0.9848484848484849
Person:  137
Transitions:  [2, 3]
GT transitions:  0
Transitions captured:  0

A,P,R:  0.9024390243902439 0.9024390


A,P,R:  0.9615384615384616 0.9615384615384616 1.0
Num frames:  (26, 1)
Accuracy:  0.9615384615384616
0.819216157005673
Average (only transitions) A,P,R 0.7353200717558199 0.6175127740229859 0.7918057635878275
Average (all targets) A,P,R,F, ttr 0.8192161570056727 0.7490633221230135 0.8614578598368186 143.9686274509804 [5, 5, 5, 3, 5, 1, 1, 3, 3, 3, 5, 3, 3, 1, 3, 3, 5, 5, 5, 5, 3, 5, 3, 3, 3, 5, 7, 5, 5, 5, 5, 3, 5, 5, 5, 3, 7, 5, 3, 3, 9, 7, 5, 5, 3, 3, 5, 3, 5, 5, 5, 5, 3, 5, 3, 5, 3, 5, 3, 5, 7, 7, 3, 3, 5, 5, 3, 3, 7, 3, 5, 5, 3, 3, 3, 5, 3, 3, 3, 3, 5, 3, 5, 7, 3, 3, 5, 7, 3, 3, 5, 3, 3, 3, 5, 3, 5, 7, 3, 3, 3, 5, 3, 5, 5, 5, 5, 3, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 15, 1, 3, 3, 1, 1, 1, 1, 1, 1, 17, 1, 5, 9, 1, 1, 1, 3, 3, 1, 3, 7, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 7, 3, 5, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 7, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 7, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,

In [35]:
all_metric_values

[(301,
  0.8274151915663638,
  0.7636172633050838,
  0.8480736011110701,
  0.7850096116446292,
  0.5239247311827958,
  0.5019455252918288),
 (321,
  0.8672509887341174,
  0.8215966174292079,
  0.8865660678447354,
  0.8379054501242941,
  0.8225806451612904,
  0.8015564202334631),
 (341,
  0.841795018447188,
  0.7828446295932865,
  0.8731037307603731,
  0.8046796239544459,
  0.6758064516129032,
  0.6536964980544747),
 (361,
  0.8288249060605348,
  0.769731088859597,
  0.8198433141162722,
  0.7730776725666328,
  0.45833333333333326,
  0.42023346303501946),
 (381,
  0.8124289670092041,
  0.7346457331337232,
  0.8031724913238699,
  0.7484139433740714,
  0.6384408602150539,
  0.6147859922178989),
 (401,
  0.819931087100555,
  0.7597299145558427,
  0.7887406862980397,
  0.7563218694189414,
  0.5134792626728111,
  0.48249027237354086),
 (421,
  0.8081911196854118,
  0.7712364262242261,
  0.7859624343537118,
  0.756266881285329,
  0.43483102918586786,
  0.3968871595330739),
 (441,
  0.824968484

In [36]:
np.save('./all_metric_values_db2_5steps', all_metric_values)

In [37]:
all_metric_values_cat = {}
all_metric_values_cat['all_metric_values'] = np.stack(all_metric_values)
spio.savemat('../../8tb/abstraction/unsup/all_metric_values_db2_5steps.mat', all_metric_values_cat)

### Test function

In [ ]:
backup_fname = '/media/win/HRLhkl/Q_CamSel_3L_l4_st200_db3_1tCont_2'
hkl.dump([[episode_reward, running_reward]], backup_fname+'_variables.hkl')

In [1]:
1/np.log(600*12)

NameError: name 'np' is not defined

In [ ]:
pp = 1
np.max(pTest[pp][1:,1] - pTest[pp][0:-1,1])